In [76]:
from elasticsearch_dsl import connections, Index, analyzer, DocType, Date, Text, Keyword, Search, Nested, InnerDoc
import json
import pprint
import os

In [2]:
# in case of problsems with "only allow delete api" then:
# curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_all/_settings -d '{"index.blocks.read_only_allow_delete": null}'

Defines a default connection that will be used globally

In [3]:
connections.create_connection(hosts=['localhost'], timeout=20) 

<Elasticsearch([{'host': 'localhost'}])>

Defines an index

In [4]:
judgments = Index(name = 'judgments')

In [130]:
# TODO: resolve what should be datatype for judges
class Judge(InnerDoc):
    function = Text()
    name = Text()
    specialRoles = Text()


class Judgments(DocType):
    content = Text()
    date = Date()
    title = Keyword()
    judges = Nested(Judge)
    
    class Meta:
        index = 'title'
    
    def add_judge(self, function, name, specialRoles):
        self.judges.append(
            Judge(function=function, name=name, specialRoles=specialRoles))
    

In [131]:
judgments.doc_type(JudgmentsMaping)

__main__.JudgmentsMaping

In [132]:
default_analyzer = analyzer('default_analyzer',
                           tokenizer='standard')

In [134]:
judgments.analyzer(default_analyzer)

In [135]:
judgments.delete(ignore=404)
judgments.create()

In [159]:
def read_file(file_path):
    with open (file_path) as file:
        json_content = json.load(file)
        for judgment in json_content:
            content = judgment['textContent']
            date = judgment['judgmentDate']
            title = judgment['courtCases'][0]['caseNumber']
            judges  = judgment['judges']
            # pprint.pprint(judges)
            maping = Judgments(content=content, date=date, title=title)
            for judge in judges:
                maping.add_judge(function=judge['function'], 
                             name=judge['name'], 
                             specialRoles=judge['specialRoles'])
                maping.save()
#             if not judges:
#                 maping.add_judge(function="", name="", specialRoles="")
#                 maping.save()

In [160]:
def read_all_judgments_from_2018():
    for filename in os.listdir("../data_filtered/"):
        read_file("../data_filtered/" + filename)

In [161]:
read_all_judgments_from_2018()

In [162]:
query = Search().query("match", date="2018-01-02")
response = query.execute()
print (query.to_dict())


{'query': {'match': {'date': '2018-01-02'}}}


In [164]:
for hit in query:
#     if "judges" in hit:
    pprint.pprint(hit)
#     pprint.pprint(hit.judges)

<Hit(rulings/doc/eCUVBmIBslyHz7tZ21w3): {'content': '<p>Warszawa, dnia 12 grudnia 2017 r.</p>\n    <...}>
<Hit(rulings/doc/ViUbBmIBslyHz7tZOGBZ): {'content': '<p>Warszawa, dnia 12 grudnia 2017 r.</p>\n    <...}>
<Hit(rulings/doc/BSXtBWIBslyHz7tZWFwT): {'content': '<p>Sygn. akt VI GC 889/17</p>\n    <div>\n     ...}>
<Hit(rulings/doc/wyUbBmIBslyHz7tZl2Nf): {'content': '<p>\n<strong>\n<!-- -->Sygn. akt: X C 1992/17 <...}>
<Hit(rulings/doc/xiUbBmIBslyHz7tZl2Pv): {'content': '<p>Sygn. akt VI GC 1015/16</p>\n    <div>\n    ...}>
<Hit(rulings/doc/NCUbBmIBslyHz7tZtWSu): {'content': '<p>Warszawa, dnia 12 grudnia 2017 r.</p>\n    <...}>
<Hit(title/doc/TDiUCWIBgI6iz7SfrsPn): {'content': '<p>Sygn. akt IV P 156/17</p>\n    <div>\n      ...}>
<Hit(title/doc/TTiUCWIBgI6iz7SfrsPt): {'content': '<p>Sygn. akt I C 738/17 upr.</p>\n    <div>\n  ...}>
<Hit(title/doc/TjiUCWIBgI6iz7SfrsPx): {'content': '<p>Sygn. akt I Ns 514/15</p>\n    <div>\n      ...}>
<Hit(title/doc/UTiUCWIBgI6iz7Sfr8MP): {'content': '<